In [1]:
import sys, pickle, ast
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [2]:
problem_folder = '../problems/WSCP_time_dependent'

# Load data

In [3]:
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from utils.EM_algorithm import build_EM_df, EM_algorithm
from utils.filtering import maximal_component_selection

problem_folder = project_root / 'problems' / 'WSCP_time_dependent'
is_minimization = ast.literal_eval((problem_folder / 'problem_specification' / 'minimization.txt').read_text())

candidates = list((problem_folder).glob('solvers_*.pkl'))
assert len(candidates) == 1, f"Expected 1 solvers_*.pkl, found {len(candidates)}: {candidates}"
solvers_file = candidates[0]
with open(solvers_file, 'rb') as file:
    solvers = pickle.load(file)

# Helper function

In [4]:
def run_selection(n_solvers, n_instances, n_validity_tests):
    # Sample desired number of solvers, instances, and validity tests
    indexes_solvers = np.random.choice(len(solvers), size=n_solvers, replace=True).tolist()
    reduced_to_original_id  = {j: idx for j, idx in enumerate(indexes_solvers)}
    indexes_instances = (np.random.choice(len(solvers[0]["outputs"]), size=n_instances, replace=True).tolist())
    indexes_validity_tests = (np.random.choice(len(solvers[0]["validity_tests"]), size=n_validity_tests, replace=True).tolist())

    solver_samples = []
    optimal_sampled = 0
    for i in indexes_solvers:
        solver = {}
        solver["outputs"] = [solvers[i]["outputs"][j] for j in indexes_instances]
        solver["interpretable_outputs"] = [solvers[i]["interpretable_outputs"][j] for j in indexes_instances]
        solver["validity_tests"] = [[solvers[i]["validity_tests"][l][j] for j in indexes_instances] for l in indexes_validity_tests]
        solver['correct'] = solvers[i]["correct"]
        solver['executable'] = solvers[i]["executable"]
        solver_samples.append(solver)
        optimal_sampled += solvers[i]["correct"]

    try:
        # Filter components
        S, I, T = maximal_component_selection(solver_samples)
        # Build the DataFrame for EM algorithm containing interpretable components
        df, compact_to_reduced_id = build_EM_df(solver_samples, S, I, T)
    except Exception as e:
        return [[i, -1] for i in indexes_solvers], None, optimal_sampled
    if df.empty:
        return [[i, -1] for i in indexes_solvers], None, optimal_sampled

    # Run EM algorithm
    priors = {
        "lambda": (1, 1),
        "alpha_s": (1, 1),
        "beta_s": (1, 1),
        "gamma_s": (1, 1),
        "p0": (1, 1),
        "p1": (20, 1),
    }
    nS = len(compact_to_reduced_id["solvers"])
    nI = len(compact_to_reduced_id["instances"])
    nT = len(compact_to_reduced_id["tests"])
    results_em = EM_algorithm(df, priors, nS, nI, nT, max_iter=100)

    # Compute scalarized objective
    parameters = {}

    obj_max = 1e3
    for s_id in range(len(results_em['alpha_s'])):
        reduced_id = compact_to_reduced_id["solvers"][s_id]
        for j in range(results_em['fsi'].shape[1]):
            o = solver_samples[reduced_id]["outputs"][compact_to_reduced_id["instances"][j]]
            if o["status"] == "OPTIMAL":
                obj_max = max(abs(obj_max), o["objective_value"])

    P_inf = 10 * obj_max
    P_miss = 10 * obj_max
    lambd = results_em['lambda']

    for s_id in range(len(results_em['alpha_s'])):
        alpha = results_em['alpha_s'][s_id]
        beta = results_em['beta_s'][s_id]
        gamma = results_em['gamma_s'][s_id]

        reduced_id = compact_to_reduced_id["solvers"][s_id]
        solver = solver_samples[reduced_id]
        num = 0
        den = 0
        for j in range(results_em['fsi'].shape[1]):
            o = solver["outputs"][compact_to_reduced_id["instances"][j]]
            if o["status"] == "OPTIMAL":
                num += o["objective_value"] * results_em['fsi'][s_id, j]
                den += results_em['fsi'][s_id, j]

        conditional_objective = num / den * (-1,1)[is_minimization] if den > 1e-5 else (obj_max)

        Vs = ((1-lambd) * alpha * P_inf + lambd * (beta * P_miss + (1-beta) * (gamma * conditional_objective + (1-gamma) * P_inf)))
        parameters[reduced_to_original_id[reduced_id]] = {
            "alpha": alpha,
            "beta": beta,
            "gamma": gamma,
            "conditional_objective": conditional_objective,
            "Vs": Vs
        }
    
    ranking = sorted(parameters.items(), key=lambda item: item[1]["Vs"])

    return ranking, results_em, optimal_sampled

# Run boostrapping

In [7]:
n_solvers = 50
n_instances = 50
n_validity_tests = 50
n_runs = 1000

results = {
    "optimal_rate": 0,
    "feasible_rate": 0,
    "executable_rate": 0,
    "interpretable_rate": 0,
}

for run_id in tqdm(range(n_runs)):
    ranking, results_em, optimal_sampled = run_selection(n_solvers = 50, n_instances = 50, n_validity_tests = 50)
    selected_solver = solvers[ranking[0][0]]
    results["optimal_rate"] += selected_solver["correct"] / n_runs
    results["feasible_rate"] += selected_solver["feasible"] / n_runs
    results["executable_rate"] += selected_solver["executable"] / n_runs
    results["interpretable_rate"] += selected_solver["interpretable"] / n_runs

100%|██████████| 1000/1000 [01:33<00:00, 10.74it/s]


# Report

In [10]:
print('-'*15 + ' Backbone LLM ' + '-'*16)
print(f"Optimality rate: {np.mean([solver['correct'] for solver in solvers])}")
print(f"Feasibility rate: {np.mean([solver['feasible'] for solver in solvers])}")
print(f"Executability rate: {np.mean([solver['executable'] for solver in solvers])}")
print(f"Interpretability rate: {np.mean([solver['interpretable'] for solver in solvers])}")
print('-'*15 + ' OptiHive ' + '-'*20)
print(f"Optimality rate: {results['optimal_rate']:.3f}")
print(f"Feasibility rate: {results['feasible_rate']:.3f}")
print(f"Executability rate: {results['executable_rate']:.3f}")
print(f"Interpretability rate: {results['interpretable_rate']:.3f}")
print('-'*45)

--------------- Backbone LLM ----------------
Optimality rate: 0.03
Feasibility rate: 0.12
Executability rate: 0.95
Interpretability rate: 0.95
--------------- OptiHive --------------------
Optimality rate: 0.640
Feasibility rate: 0.749
Executability rate: 1.000
Interpretability rate: 1.000
---------------------------------------------
